In [2]:
##https://medium.com/%E7%94%A8%E5%8A%9B%E5%8E%BB%E6%84%9B%E4%B8%80%E5%80%8B%E4%BA%BA%E7%9A%84%E8%A9%B1-%E5%BF%83%E4%B9%9F%E6%9C%83%E7%97%9B%E7%9A%84/%E9%BB%98%E9%BB%98%E5%9C%B0%E5%AD%B8deep-learning-4-698d00aba2ca

import os
import glob
import numpy as np
import keras
from keras.preprocessing.image import  img_to_array, load_img
from PIL import Image
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
#%%

Using TensorFlow backend.


In [3]:

dict_labels = {"daisy":0, "dandelion":1, "rose":2, "sunflower":3, "tulip":4}
size = (256,256) #由於原始資料影像大小不一，因此制定一個統一值
nbofdata=2000   #從各個資料夾中抓取特定數量的檔案
#%% Read Traget Folders' Path
labels=['daisy','dandelion','rose','sunflower','tulip']
base_path = '../data/image_data/train/'
layers_of_folders=0
folder_list=[]
if base_path :
    folder_layers=[]
    files = os.scandir(base_path)
    #  Get the 1st layer of folder
    first_folder = []
    first_folder_kind = []
    for entry in files:
        if entry.is_dir():
            first_folder.append(entry.path)
            first_folder_kind.append(entry.name)
    folder_layers.append(first_folder_kind)
    folder_list.append(first_folder)
    #  Get the 2nd layer of folder
    second_folder = []
    if first_folder:
        second_folder = []
        second_folder_kind = []
        layers_of_folders+=1
        for fldr in first_folder:
            files = os.scandir(fldr)
            for entry in files:
                if entry.is_dir():
                    second_folder.append(entry.path)
                    second_folder_kind.append(entry.name)
        second_folder_kind= second_folder_kind[0:int(len(second_folder_kind)/len(first_folder_kind))]
        folder_layers.append(second_folder_kind)
        folder_list.append(second_folder)
    #  Get the 3rd layer of folder
    third_folder = []
    if second_folder:
        third_folder = []
        third_folder_kind = []
        layers_of_folders+=1
        for fldr in second_folder:
            files = os.scandir(fldr)
            for entry in files:
                if entry.is_dir():
                    third_folder.append(entry.path)
                    third_folder_kind.append(entry.name)
        third_folder_kind= third_folder_kind[0:int(len(third_folder_kind)/(len(second_folder_kind)*len(first_folder_kind)))]
        folder_list.append(third_folder)
    #  Get the 4th layer of folder
    forth_folder = []
    if third_folder:
        forth_folder = []
        forth_folder_kind = []
        layers_of_folders+=1
        for fldr in third_folder:
            files = os.scandir(fldr)
            for entry in files:
                if entry.is_dir():
                    forth_folder.append(entry.path)
                    forth_folder_kind.append(entry.name)
        forth_folder_kind= forth_folder_kind[0:int(len(forth_folder_kind)/(len(third_folder_kind)*len(second_folder_kind)*len(first_folder_kind)))]
        folder_list.append(forth_folder)
     #  Get the 5th layer of folder
    if forth_folder:
        fifth_folder = []
        fifth_folder_kind = []
        layers_of_folders+=1
        for fldr in third_folder:
            files = os.scandir(fldr)
            for entry in files:
                if entry.is_dir():
                    fifth_folder.append(entry.path)
                    fifth_folder_kind.append(entry.name)
        fifth_folder_kind= fifth_folder_kind[0:int(len(fifth_folder_kind)/(len(forth_folder_kind)*len(third_folder_kind)*len(second_folder_kind)*len(first_folder_kind)))]
#%% Read Image Files (*.tif)
datanumber=nbofdata
blob=[]
blob_nparray=[]
image_data=[]
conc = 0
labels_dict={}
for entry1 in folder_list[layers_of_folders - 1]:
    blob = []
    cellname = os.path.basename(os.path.dirname(entry1))  # extract cell name
    print('cellname:'+ cellname)
    concnames = os.path.basename(entry1)  # extract concentration
    print('concname:' +concnames)
    if concnames in labels:
        labels_dict[conc] = concnames
        #print(entry1) 
        fnamelist = glob.glob(os.path.join(entry1, '*.jpg'))
        for filename in fnamelist[0:datanumber]:
            im = Image.open(filename)
            if im is not None:
                if im.mode=='RGB':
                    im=im.resize(size,Image.BILINEAR)
                    imarray = np.array(im)
                    blob.append(imarray)
        ind = np.reshape(np.arange(1, len(blob) + 1), (-1, 1))
        blob_nparray = np.reshape(np.asarray(blob), (len(blob), blob[1].size))
        blob_nparray = np.hstack((blob_nparray, ind, conc * np.ones((len(blob), 1))))
        image_data.append(np.asarray(blob_nparray, dtype=np.float32))
        print(concnames+'  finished!')
        conc += 1


cellname:train
concname:daisy
daisy  finished!
cellname:train
concname:dandelion
dandelion  finished!
cellname:train
concname:rose
rose  finished!
cellname:train
concname:sunflower
sunflower  finished!
cellname:train
concname:tulip
tulip  finished!


In [4]:
print(len(labels))

5


In [5]:
#%%
for j in range(len(labels)):
    trytry=image_data[j][:]
# Prepare data
    LengthT = trytry.shape[0]
    trytry_index = trytry[...,-2:-1]
    trytry_label = trytry[...,-1:] #['Nega' for x in range(lengthN*4)] #Nega_data[...,-1:]
    trytry = trytry[...,:-2]
    
    # Normalize image by subtracting mean image
    trytry -= np.reshape(np.mean(trytry, axis=1), (-1,1))
    # Reshape images
    trytry = np.reshape(trytry, (trytry.shape[0],256,256,3))
    
#    # Rotate images
#    for i in range(3):
#        trytry[LengthT*(i+1):LengthT*(i+2)] = np.rot90(trytry[:LengthT], i+1, (1,2))
    # Add channel dimension to fit in Conv2D
    trytry = trytry.reshape(-1,256,256,3)
    np.random.shuffle(trytry)
    trytry_train_upto = round(trytry.shape[0] * 8 / 10)  # 8成當訓練集
    trytry_test_upto = trytry.shape[0]                   # 2成當測試集    
    if j is 0:
        train_data = trytry[:trytry_train_upto]
        test_data = trytry[trytry_train_upto:trytry_test_upto]
        train_label = trytry_label[:trytry_train_upto]
        test_label = trytry_label[trytry_train_upto:trytry_test_upto]
        
    else:
        train_data = np.concatenate((train_data, 
                                     trytry[:trytry_train_upto]), axis=0)
        
        test_data = np.concatenate((test_data, 
                                    trytry[trytry_train_upto:trytry_test_upto]), axis=0)
        
        train_label = np.concatenate((train_label, 
                                     trytry_label[:trytry_train_upto]), axis=0)
        
        
        test_label = np.concatenate((test_label, 
                                    trytry_label[trytry_train_upto:trytry_test_upto]), axis=0)
        
test_label = keras.utils.to_categorical(test_label, num_classes=len(labels))
train_label = keras.utils.to_categorical(train_label, num_classes=len(labels))


In [6]:
print(test_data.shape[1])
print(train_data.shape[1])
print(len(test_label))
print(len(train_label))

256
256
565
2258


In [7]:

#%% Shuffle data
import random
temp = list(zip(train_data, train_label))

random.shuffle(temp)

train_data,train_label = zip(*temp)

train_data=np.asarray(train_data)
train_label=np.asarray(train_label)


In [8]:
print(train_data)

[[[[   7.390915      7.390915      7.390915  ]
   [   8.390915      8.390915      8.390915  ]
   [   9.390915      9.390915      9.390915  ]
   ...
   [   2.390915      4.390915      3.390915  ]
   [   0.39091492    2.390915      1.3909149 ]
   [  -1.6090851     0.39091492   -0.6090851 ]]

  [[   8.390915      8.390915      8.390915  ]
   [   9.390915      9.390915      9.390915  ]
   [  10.390915     10.390915     10.390915  ]
   ...
   [   3.390915      5.390915      4.390915  ]
   [   1.3909149     3.390915      2.390915  ]
   [  -0.6090851     1.3909149     0.39091492]]

  [[  10.390915     10.390915     10.390915  ]
   [  11.390915     11.390915     11.390915  ]
   [  11.390915     11.390915     11.390915  ]
   ...
   [   2.390915      5.390915      4.390915  ]
   [   2.390915      4.390915      3.390915  ]
   [   0.39091492    2.390915      1.3909149 ]]

  ...

  [[ -35.609085    -39.609085    -42.609085  ]
   [ -33.609085    -35.609085    -39.609085  ]
   [ -33.609085    -34.609

In [ ]:

#%% VGG 16 only for classification
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop
# Generate model
model = Sequential()
# input: 190x190 images with 3 channels -> (190, 190, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(256,256,3),padding='same',name='block1_conv2_1'))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same',name='block1_conv2_2'))
model.add(MaxPooling2D(pool_size=(2, 2),name='block1_MaxPooling'))

model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu',padding='same',name='block2_conv2_1'))
model.add(Conv2D(128, (3, 3), activation='relu',padding='same',name='block2_conv2_2'))
model.add(MaxPooling2D(pool_size=(2, 2),name='block2_MaxPooling'))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), activation='relu',padding='same',name='block3_conv2_1'))
model.add(Conv2D(256, (3, 3), activation='relu',padding='same',name='block3_conv2_2'))
model.add(Conv2D(256, (3, 3), activation='relu',padding='same',name='block3_conv2_3'))
model.add(MaxPooling2D(pool_size=(2, 2),name='block3_MaxPooling'))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), activation='relu',padding='same',name='block4_conv2_1'))
model.add(Conv2D(512, (3, 3), activation='relu',padding='same',name='block4_conv2_2'))
model.add(Conv2D(512, (3, 3), activation='relu',padding='same',name='block4_conv2_3'))
model.add(MaxPooling2D(pool_size=(2, 2),name='block4_MaxPooling'))
model.add(Dropout(0.25))
#
#model.add(Conv2D(512, (3, 3), activation='relu',padding='same',name='block5_conv2_1'))
#model.add(Conv2D(512, (3, 3), activation='relu',padding='same',name='block5_conv2_2'))
#model.add(Conv2D(512, (3, 3), activation='relu',padding='same',name='block5_conv2_3'))
#model.add(MaxPooling2D(pool_size=(2, 2),name='block5_MaxPooling'))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu',name='final_output_1'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu',name='final_output_2'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='sigmoid',name='class_output'))  #OUTPUT LAYER 
optimizer = RMSprop(lr=1e-4)
objective = 'binary_crossentropy'
model.compile(loss=objective, optimizer=optimizer, metrics=['accuracy'])
EStop = EarlyStopping(monitor='val_acc', min_delta=0, 
                      patience=10, verbose=1, mode='auto')
#%% Start Traning Model
history = model.fit(train_data, train_label, batch_size=4, epochs=35 ,shuffle=True, validation_split=0.2,callbacks=[EStop])
#%%
model.save('catdog_model1228256.h5') 
predictions=model.predict(test_data)


Train on 1806 samples, validate on 452 samples
Epoch 1/35
1806/1806 [==============================] - 334s 185ms/step - loss: 0.6131 - accuracy: 0.7555 - val_loss: 0.5210 - val_accuracy: 0.8000
Epoch 2/35


C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


1806/1806 [==============================] - 371s 205ms/step - loss: 0.5093 - accuracy: 0.7831 - val_loss: 0.4634 - val_accuracy: 0.7996
Epoch 3/35
1806/1806 [==============================] - 494s 274ms/step - loss: 0.4752 - accuracy: 0.7897 - val_loss: 0.4340 - val_accuracy: 0.8062
Epoch 4/35
1806/1806 [==============================] - 599s 332ms/step - loss: 0.4482 - accuracy: 0.7998 - val_loss: 0.3920 - val_accuracy: 0.8168
Epoch 5/35
1806/1806 [==============================] - 648s 359ms/step - loss: 0.4294 - accuracy: 0.8116 - val_loss: 0.4038 - val_accuracy: 0.8296
Epoch 6/35
1806/1806 [==============================] - 780s 432ms/step - loss: 0.4114 - accuracy: 0.8179 - val_loss: 0.3562 - val_accuracy: 0.8465
Epoch 7/35
1804/1806 [============================>.] - ETA: 0s - loss: 0.3981 - accuracy: 0.8307

In [ ]:
# 載入模型
model = keras.models.load_model('catdog_model1228256.h5') 

In [ ]:
import tensorflow as tf
import keras
# 準備 x_test 與 y_test 資料 ... [略]
size = (256, 256)
nbofdata=2000
base_path = '../data/image_data/test/'
layers_of_folders=0
folder_list=[]    
labels=['test']

if base_path :
    folder_layers=[]
    files = os.scandir(base_path)
    #  Get the 1st layer of folder
    first_folder = []
    first_folder_kind = []
    for entry in files:
        if entry.is_dir():
            first_folder.append(entry.path)
            first_folder_kind.append(entry.name)
    folder_layers.append(first_folder_kind)
    folder_list.append(first_folder)

In [ ]:
datanumber=nbofdata
blob=[]
blob_nparray=[]
image_data=[]
conc = 0
fc = 0
labels_dict={}
fn = {}
for entry1 in folder_list[layers_of_folders - 1]:
    blob = []
    cellname = os.path.basename(os.path.dirname(entry1))  # extract cell name
    # print(cellname)
    concnames = os.path.basename(entry1)  # extract concentration
    # print(concnames)
    if concnames in labels:
        labels_dict[conc] = concnames
        fnamelist = glob.glob(os.path.join(entry1, '*.jpg'))
        for filename in fnamelist[0:datanumber]:
            im = Image.open(filename)
            if im is not None:
                if im.mode=='RGB':
                    im=im.resize(size,Image.BILINEAR)
                    imarray = np.array(im)
                    blob.append(imarray)
                    fn[fc] = filename
                    fc += 1
        ind = np.reshape(np.arange(1, len(blob) + 1), (-1, 1))
        blob_nparray = np.reshape(np.asarray(blob), (len(blob), blob[1].size))
        blob_nparray = np.hstack((blob_nparray, ind, conc * np.ones((len(blob), 1))))
        image_data.append(np.asarray(blob_nparray, dtype=np.float32))
        print(ind)
        print(blob_nparray)
        print(concnames+'  finished!')
        conc += 1

In [ ]:
print(fn[0])

In [ ]:
sp = {}
for i in range(2000):
    sp[i] = fn[i].split('/')
    sp[i] = sp[i][4].split('test\\')
    sp[i] = sp[i][1].split('.jpg')

In [ ]:
for i in range(10):
    print(sp[i][0])

In [ ]:
ids = {}
for i in range(2000):
    ids[i] = sp[i][0]

In [ ]:
for j in range(len(labels)):
    trytry=image_data[j][:]
# Prepare data
    LengthT = trytry.shape[0]

    trytry_index = trytry[...,-2:-1]

    trytry_label = trytry[...,-1:] #['Nega' for x in range(lengthN*4)] #Nega_data[...,-1:]

    trytry = trytry[...,:-2]

    # Normalize image by subtracting mean image
    trytry -= np.reshape(np.mean(trytry, axis=1), (-1,1))
    # Reshape images
    trytry = np.reshape(trytry, (trytry.shape[0],256,256,3))
    
#    # Rotate images
#    for i in range(3):
#        trytry[LengthT*(i+1):LengthT*(i+2)] = np.rot90(trytry[:LengthT], i+1, (1,2))
    # Add channel dimension to fit in Conv2D
    trytry = trytry.reshape(-1,256,256,3)
    trytry_test_upto = trytry.shape[0]
    if j is 0:
        test_data = trytry[:trytry_test_upto]      
    else:     
        test_data = np.concatenate((test_data, 
                                    trytry[trytry_train_upto:trytry_test_upto]), axis=0)

In [ ]:
test_data.shape

In [ ]:
predictions=model.predict(test_data)

In [ ]:
for i in range(10):
    for j in range(5):
        print(predictions[i][j])

In [ ]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import array_to_img

plt.imshow(array_to_img(test_data[1]))
plt.show()

In [ ]:
cnn_pred = {}
for i in range(2000):
    for j in range(5):
        if predictions[i][j] == np.max(predictions[i]):
            if j == 0:
                cnn_pred[i] = 0
            if j == 1:
                cnn_pred[i] = 1
            if j == 2:
                cnn_pred[i] = 2
            if j == 3:
                cnn_pred[i] = 3
            if j == 4:
                cnn_pred[i] = 4

In [ ]:
cnn_pred

In [ ]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

submit = pd.DataFrame({'id': ids, 'flower_class': cnn_pred})
header = ["id", "flower_class"]
submit.to_csv('cnn_resnet_JIMMY2.csv', columns = header, index=False)